(ref:lec3:ex3)=
# Exercise 3.3

{ilo}`analysis_closed_loop`

```{figure} res/ex3.svg
---
height: 180
name: fig:lec3:ex3:block
---
Block diagram of the closed loop
```

A system $G(s)$ is controlled using a feedback with a proportional controller according to {numref}`fig:lec3:ex3:block`. $G(s)$ has **no unstable pole**, that is, no pole with positive real part.

In {numref}`fig:lec3:ex3:nyquist`, the Nyquist diagram of four different systems $G(s)$ are reported. Find, for each of these systems, the values of $K$ for which the closed-loop is stable. 


```{figure} res/ex3-nyq.png
---
width: 700
name: fig:lec3:ex3:nyquist
---
Nyquist plots of $G(s)$
```